# Stable Diffusion数据集下的情况

In [1]:
import pandas as pd
compare_promts = pd.read_parquet('blip2_ft_sd/result/rl_other_generative_model_test.parquet')
# blip2_ft_sd/result/blip2_0.2_10_test.parquet
compare_promts

,generated_caption,reference_prompt,img_path,rl_generation
0,a beautiful fantasy digital painting of a Nors...,a beautiful Norse Warrior based on Hades with ...,./imgs/image_2700.png,"Greg Rutkowski and Raymond Swanland, a fantasy..."
1,"a blueprints of new york city, concept art, co...","blueprints for New York City, concept art, blu...",./imgs/image_4113.png,"a blueprints of new york city, concept art, by..."
2,"a spider with eyes of blood, concept art, high...","d d monster, huge spider monster covered in ey...",./imgs/image_3278.png,"a spider with eyes of blood, highly detailed, ..."
3,"concept art of medieval knight armor, 4 k, hig...","different views of medieval knights, beautiful...",./imgs/image_3909.png,"Concept art of medieval knight armor, 4 k, hig..."
4,lofi underwater bioshock steampunk portrait of...,"lofi underwater steampunk bioshock bikini, oct...",./imgs/image_2164.png,Portrait lofi underwater bioshock steampunk po...
...,...,...,...,...
295,"a beautiful painting of a medieval knight, by ...",a robot wearing medieval armor in the style of...,./imgs/image_4086.png,a painting of a medieval knight who looks like...
296,steampunk portrait of a man with a top hat and...,"Lofi Steampunk portrait, Pixar style, by Trist...",./imgs/image_3450.png,a steampunk portrait of a top sleazy man weari...
297,a beautiful painting of a dinosaur in a candyl...,"A dinosaur in a candy forest, digital art, tre...",./imgs/image_2398.png,a beautiful painting of a dinosaur in a candyl...
298,"a beautiful painting of a cute cat smiling, bl...",cute blue striped cat of cheshire from alice i...,./imgs/image_1770.png,A cute blue cat in the background smiling as a...


## 根据不同的prompt生成图片

In [2]:
from diffusers import StableDiffusionPipeline
from diffusers import PNDMScheduler
import torch
from PIL import Image


device = "cuda" if torch.cuda.is_available() else "cpu"


model_id = "runwayml/stable-diffusion-v1-5"
scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")

weight_dtype = torch.float16

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=weight_dtype)
pipe = pipe.to(device)
image_length = 512

/home/fenghe/anaconda3/envs/ldm/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## Bert

In [3]:
from bert_score import score

def bert_sim_eval(candidate_sentence, target_sentence):
    # 将句子放入列表中，因为 bert-score 的 score 函数期望列表输入
    candidates = [candidate_sentence]
    references = [target_sentence]

    # 计算 BertScore
    P, R, F1 = score(candidates, references, lang='en', verbose=True)
    return {"P":P.item(),"R":R.item(),"F1":F1.item()}

In [4]:
from tqdm import tqdm
P_ours = []
R_ours = []
F1_ours = []
for index,item in tqdm(compare_promts.iterrows(),total=len(compare_promts)):    
    res = bert_sim_eval(item['rl_generation'],item['reference_prompt'])
    P_ours.append(res['P'])
    R_ours.append(res['R'])
    F1_ours.append(res['F1'])

  0%|          | 0/300 [00:00<?, ?it/s]Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 300/300 [04:36<00:00,  1.08it/s]

done in 0.02 seconds, 48.50 sentences/sec


In [5]:
print(sum(P_ours)/len(P_ours))
print(sum(R_ours)/len(R_ours))
print(sum(F1_ours)/len(F1_ours))

0.8548540172974268
0.8382853126525879
0.8462189211448033


## CLIP: 目标图片与两种方式的生成Prompt产生的图片进行计算

In [6]:
import open_clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, _, clip_preprocess = open_clip.create_model_and_transforms("ViT-L-14", pretrained="openai", device=device)

def measure_clip_imgtxt_similarity(image_path, txt_1):
    text = open_clip.tokenize([txt_1]).to(device)
    orig_images_t = clip_preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = clip_model.encode_image(orig_images_t)
        text_features = clip_model.encode_text(text)
        
    cosine_similarity = torch.nn.functional.cosine_similarity(image_features, text_features, dim=1)

    return cosine_similarity.cpu().numpy().tolist()

    

def measure_clip_imgs_similarity(orig_images_t, pred_imgs_t, clip_model):
    with torch.no_grad():
        orig_feat = clip_model.encode_image(orig_images_t)
        orig_feat = orig_feat / orig_feat.norm(dim=1, keepdim=True)

        pred_feat = clip_model.encode_image(pred_imgs_t)
        pred_feat = pred_feat / pred_feat.norm(dim=1, keepdim=True)
        return (orig_feat @ pred_feat.t()).mean().item()

In [7]:
from diffusers import StableDiffusionPipeline
from diffusers import PNDMScheduler
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


model_id = "runwayml/stable-diffusion-v1-5"
scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    scheduler=scheduler,
    torch_dtype=torch.float16,
    ).to(device)

image_length = 512

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
from tqdm import tqdm

eval_sim_ours = []

for index, item in tqdm(compare_promts.iterrows(),total=len(compare_promts)):
    orig_image = Image.open(item['img_path']).convert('RGB')
    with torch.no_grad():
        pred_imgs = pipe(
                    item['rl_generation'],
                    num_images_per_prompt=1,
                    guidance_scale=7.5,
                    num_inference_steps=50,
                    height=image_length,
                    width=image_length,
                    ).images
        orig_images_temp = [clip_preprocess(orig_image).unsqueeze(0)]
        orig_images_t = torch.cat(orig_images_temp).to(device)
        pred_imgs_temp = [clip_preprocess(i).unsqueeze(0) for i in pred_imgs]
        pred_imgs_t = torch.cat(pred_imgs_temp).to(device)
        eval_sim_ours.append(measure_clip_imgs_similarity(orig_images_t, pred_imgs_t, clip_model))

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 1/300 [00:02<11:05,  2.23s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 2/300 [00:04<10:19,  2.08s/it]

...

 99%|█████████▉| 298/300 [10:15<00:04,  2.07s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|█████████▉| 299/300 [10:17<00:02,  2.07s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 300/300 [10:19<00:00,  2.06s/it]


In [9]:
sum(eval_sim_ours)/len(eval_sim_ours)

0.7052789385120074

## LPIPS： 目标图像与生成图像之间的语义相似度

In [10]:
import torch
import lpips
import itertools
from PIL import Image
from torchvision import transforms

# 加载 LPIPS 模型
lpips_model = lpips.LPIPS(net='alex').to(device)

# 加载图像并预处理
transform = transforms.Compose([transforms.Resize((256, 256)),transforms.ToTensor(),])

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

def measure_lpips_imgs_similarity(img1, img2):
    distance = lpips_model(img1, img2)
    # 计算多样性指标，这里我们取平均值
    return distance.item()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/fenghe/anaconda3/envs/ldm/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


/home/fenghe/anaconda3/envs/ldm/lib/python3.9/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map

In [11]:
lpips_sim_ours = []

for index, item in tqdm(compare_promts.iterrows(),total=len(compare_promts)):
    orig_image_t = load_image(item['img_path']).to(device)
    with torch.no_grad():
        pred_imgs = pipe(
                    item['rl_generation'],
                    num_images_per_prompt=1,
                    guidance_scale=7.5,
                    num_inference_steps=50,
                    height=image_length,
                    width=image_length,
                    )
        pred_img = pred_imgs.images[0]
        pred_img_t = transform(pred_img).unsqueeze(0).to(device)

        lpips_sim_ours.append(lpips_model(orig_image_t, pred_img_t).item())
        #.append(measure_lpips_imgs_similarity())

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 1/300 [00:01<09:56,  1.99s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

  1%|          | 2/300 [00:03<09:49,  1.98s/it]

...

 99%|█████████▉| 298/300 [10:07<00:04,  2.04s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|█████████▉| 299/300 [10:09<00:02,  2.04s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 300/300 [10:11<00:00,  2.04s/it]


In [12]:
print(sum(lpips_sim_ours)/len(lpips_sim_ours))

0.6048574003577233
